# Práctica 3: Optimización combinatoria

En este *notebook* se aplica el código en C++ de los algoritmos de la práctica para que puedan ser replicados en distintos ordenadores.


Autores:




In [2]:
# Imports
from google.colab import files
import matplotlib.pyplot as plt
from scipy import optimize
import math
import numpy as np


## Algoritmo de profundidad iterativo

### Básico

Carga del código **TSP_DFS.cpp**

In [ ]:
files.upload()

In [ ]:
%%shell

g++ TSP_DFS.cpp -o TSP_DFS.o

### Con poda

Carga del código **TSP_DFS_poda.cpp**

In [ ]:
files.upload()

In [ ]:
%%shell

g++ TSP_DFS_poda.cpp -o TSP_DFS_poda.o

## Algoritmo de búsqueda local

Carga del código **TspLocal.cpp**

In [ ]:
files.upload()

In [ ]:
%%shell

g++ TSPLocal.cpp -o TSPLocal.o

## Coste computacional

Cargar los mapas subiendo la carpeta comprimida **mapas.zip**

In [ ]:
files.upload()

In [ ]:
!unzip mapas.zip

### Cálculo empírico del TSP DFS

A continuación, se prueba el algoritmo en diversos mapas generados aleatoriamente, los cuales tienen tamaños distintos que van de 3 a 13 ciudades.

In [ ]:
%%shell 
rm DFS_output.txt

for f in mapas/*; do
  echo "Mapa: $f" >> DFS_output.txt
  size="$(cut -d'-' -f1 <<< $f | cut -d't' -f2)"
  ./TSP_DFS.o $f $size 0 >> DFS_output.txt
  echo "" >> DFS_output.txt
done

**Gráfica evolución coste**

In [ ]:
DFS_data_s = {}
DFS_data_ns = {}
size = 0
simmetric = False

with open("DFS_output.txt", 'r') as f:
  for line in f.readlines():
    if line.split(":")[0] == "Mapa":
      size = line.split("t")[1].split("-")[0]
      simmetric = line.split("-")[1][0] == "s"

    if line.split(":")[0] == "Tiempo":
      time_path = float(line.split(": ")[1][:-1])
      
      if simmetric:
        if size in DFS_data_s:
          DFS_data_s[size].append(time_path)
        else:
          DFS_data_s[size] = [time_path]
        
      else:
        if size in DFS_data_ns:
          DFS_data_ns[size].append(time_path)
        else:
          DFS_data_ns[size] = [time_path]

In [ ]:
x_s = []
y_s = []
x_ns = []
y_ns = []

for key in DFS_data_s.keys():
  for value in DFS_data_s[key]:
    x_s.append(int(key))
    y_s.append(value)

for key in DFS_data_ns.keys():
  for value in DFS_data_ns[key]:
    x_ns.append(int(key))
    y_ns.append(value)

In [ ]:
plt.figure(figsize=(10,5))
plt.scatter(x_s, y_s, s=60, label="Mapas simétricos")
plt.scatter(x_ns, y_ns, s=20, label="Mapas no simétricos")
plt.xticks([x for x in range(3, 13)])
plt.xlabel("Tamaño mapas")
plt.ylabel("Tiempo (s)")
plt.title("Gráfica comparativa de tiempo entre mapas simétricos y no simétricos", fontweight="bold")
plt.legend()
plt.show()

In [ ]:
def DFS_cost(n, a):
  cost = np.zeros_like(n)
  for k, num in enumerate(n):
    for i in range(int(num)-1):
      cost[k] += math.factorial(num-1)/math.factorial(i)
    cost[k] *= num
  return a * cost 

In [ ]:
params, params_covariance = optimize.curve_fit(DFS_cost, np.array(x_s + x_ns), np.array(y_s + y_ns))
params

In [ ]:
plot_orderd = list(zip(x_s + x_ns, DFS_cost(np.array(x_s + x_ns), params[0])))
plot_orderd = sorted(plot_orderd, key=lambda x: x[0])

x, y = zip(*plot_orderd)

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(x, y, c='r', label='Valores Esperados', lw=2)
plt.scatter(x_s + x_ns, y_s + y_ns, s=60, label="Valores Empíricos")

plt.xticks([x for x in range(3, 13)])
plt.xlabel("Tamaño mapas")
plt.ylabel("Tiempo (s)")
plt.title("Gráfica comparativa de valores empíricos y analíticos", fontweight="bold")
plt.legend()
plt.show()

### Cálculo empírico del TSP DFS poda


In [ ]:
%%shell 
rm DFS_poda_output.txt

for f in mapas/*; do
  echo "Mapa: $f" >> DFS_poda_output.txt
  size="$(cut -d'-' -f1 <<< $f | cut -d't' -f2)"
  ./TSP_DFS_poda.o $f $size 0 >> DFS_poda_output.txt
  echo "" >> DFS_poda_output.txt
done

**Gráfica evolución coste**

In [102]:
DFS_poda_data_s = {}
DFS_poda_data_ns = {}
size = 0
simmetric = False

with open("DFS_poda_output.txt", 'r') as f:
  for line in f.readlines():
    if line.split(":")[0] == "Mapa":
      size = line.split("t")[1].split("-")[0]
      simmetric = line.split("-")[1][0] == "s"

    if line.split(":")[0] == "Tiempo":
      time_path = float(line.split(": ")[1][:-1])
      
      if simmetric:
        if size in DFS_poda_data_s:
          DFS_poda_data_s[size].append(time_path)
        else:
          DFS_poda_data_s[size] = [time_path]
        
      else:
        if size in DFS_poda_data_ns:
          DFS_poda_data_ns[size].append(time_path)
        else:
          DFS_poda_data_ns[size] = [time_path]

In [103]:
x_s = []
y_s = []
x_ns = []
y_ns = []

for key in DFS_poda_data_s.keys():
  for value in DFS_poda_data_s[key]:
    x_s.append(int(key))
    y_s.append(value)

for key in DFS_poda_data_ns.keys():
  for value in DFS_poda_data_ns[key]:
    x_ns.append(int(key))
    y_ns.append(value)

In [ ]:
plt.figure(figsize=(10,5))
plt.scatter(x_s, y_s, s=60, label="Mapas simétricos")
plt.scatter(x_ns, y_ns, s=20, label="Mapas no simétricos")
plt.xticks([x for x in range(3, 20)])
plt.xlabel("Tamaño mapas")
plt.ylabel("Tiempo (s)")
plt.title("Gráfica comparativa de tiempo entre mapas simétricos y no simétricos", fontweight="bold")
plt.legend()
plt.show()

### Cálculo empírico del TSP Local

In [ ]:
%%shell 
rm TSPLocal_output.txt

for f in mapas/*; do
  echo "Mapa: $f" >> TSPLocal_output.txt
  size="$(cut -d'-' -f1 <<< $f | cut -d't' -f2)"
  ./TSPLocal.o $f $size 0 >> TSPLocal_output.txt
  echo "" >> TSPLocal_output.txt
done

**Gráfica evolución coste**

In [9]:
TSPLocal_data_s = {}
TSPLocal_data_ns = {}
size = 0
simmetric = False

with open("TSPLocal_output.txt", 'r') as f:
  for line in f.readlines():
    if line.split(":")[0] == "Mapa":
      size = line.split("t")[1].split("-")[0]
      simmetric = line.split("-")[1][0] == "s"

    if line.split(":")[0] == "Tiempo":
      time_path = float(line.split(": ")[1][:-1])
      
      if simmetric:
        if size in TSPLocal_data_s:
          TSPLocal_data_s[size].append(time_path)
        else:
          TSPLocal_data_s[size] = [time_path]
        
      else:
        if size in TSPLocal_data_ns:
          TSPLocal_data_ns[size].append(time_path)
        else:
          TSPLocal_data_ns[size] = [time_path]

In [10]:
x_s = []
y_s = []
x_ns = []
y_ns = []

for key in TSPLocal_data_s.keys():
  for value in TSPLocal_data_s[key]:
    x_s.append(int(key))
    y_s.append(value)

for key in TSPLocal_data_ns.keys():
  for value in TSPLocal_data_ns[key]:
    x_ns.append(int(key))
    y_ns.append(value)

In [ ]:
plt.figure(figsize=(10,5))
plt.scatter(x_s, y_s, s=60, label="Mapas simétricos")
plt.scatter(x_ns, y_ns, s=20, label="Mapas no simétricos")
plt.xticks([x for x in range(3, 20)])
plt.xlabel("Tamaño mapas")
plt.ylabel("Tiempo (s)")
plt.title("Gráfica comparativa de tiempo entre mapas simétricos y no simétricos", fontweight="bold")
plt.legend()
plt.show()

In [12]:
def local_cost(n, a):
  cost = np.zeros_like(n)
  for k, num in enumerate(n):
    cost[k] += (num**2 + num) * math.factorial(num-1)
  return a * cost 

In [27]:
def local_cost(n, a):
  return a * n**3

In [ ]:
params, params_covariance = optimize.curve_fit(local_cost, np.array(x_s + x_ns), np.array(y_s + y_ns))
params

In [29]:
plot_orderd = list(zip(x_s + x_ns, local_cost(np.array(x_s + x_ns), params[0])))
plot_orderd = sorted(plot_orderd, key=lambda x: x[0])

x, y = zip(*plot_orderd)

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(x, y, c='r', label='Valores Esperados', lw=2)
plt.scatter(x_s + x_ns, y_s + y_ns, s=60, label="Valores Empíricos")

plt.xticks([x for x in range(3, 20)])
plt.xlabel("Tamaño mapas")
plt.ylabel("Tiempo (s)")
plt.title("Gráfica comparativa de valores empíricos y analíticos", fontweight="bold")
plt.legend()
plt.show()

### Calidad de la solución

In [36]:
DFS_poda_cost = {}
name = ""

with open("DFS_poda_output.txt", 'r') as f:
  for line in f.readlines():
    if line.split(":")[0] == "Mapa":
      name = line.split(": ")[1][:-1]

    if line.split(":")[0] == "Coste":
      cost = int(line.split(": ")[1][:-1])
      DFS_poda_cost[name] = cost

In [38]:
TSPLocal_cost = {}
name = ""

with open("TSPLocal_output.txt", 'r') as f:
  for line in f.readlines():
    if line.split(":")[0] == "Mapa":
      name = line.split(": ")[1][:-1]

    if line.split(":")[0] == "Coste":
      cost = int(line.split(": ")[1][:-1])
      TSPLocal_cost[name] = cost

In [40]:
comp = []

for key in TSPLocal_cost.keys():
  if key in DFS_poda_cost and key in TSPLocal_cost:
    comp.append(DFS_poda_cost[key]/TSPLocal_cost[key])

In [ ]:
print("La calidad media de las soluciones es " + str((sum(comp)/len(comp)*100)) + "%")